# Download Spacy e Arquivos

In [1]:
!python -m spacy download pt_core_news_sm

     |████████████████████████████████| 21.2 MB 1.4 MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-py3-none-any.whl size=21186281 sha256=341203cce94d99f4f51c619d3af429bc90571528b039f491ce650ecbb612340f
  Stored in directory: /tmp/pip-ephem-wheel-cache-2fd53vgr/wheels/c3/f9/0c/5c014a36941a00f5df5fc0756cb961d7c457a978e697a6ce3b
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [2]:
!curl  http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s300.zip -O > cbow_s300.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  886M  100  886M    0     0  7656k      0  0:01:58  0:01:58 --:--:-- 6205k


In [3]:
!curl  http://143.107.183.175:22980/download.php?file=embeddings/word2vec/skip_s300.zip -O  > skip_s300.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  914M  100  914M    0     0  10.4M      0  0:01:27  0:01:27 --:--:-- 11.2M


In [4]:
!unzip skip_s300.zip
!unzip cbow_s300.zip

Archive:  skip_s300.zip
  inflating: skip_s300.txt           
Archive:  cbow_s300.zip
  inflating: cbow_s300.txt           


## Spacy e Import 

In [5]:
import pandas as pd
import spacy

In [6]:
nlp = spacy.load("pt_core_news_sm")

In [7]:
uri_treino = 'https://caelum-online-public.s3.amazonaws.com/1638-word-embedding/treino.csv'
uri_teste = 'https://caelum-online-public.s3.amazonaws.com/1638-word-embedding/teste.csv'
dados_treino = pd.read_csv(uri_treino)
dados_teste = pd.read_csv(uri_teste)
dados_treino.sample(5)

,title,text,date,category,subcategory,link
74136,Ex-sócios da XP se unem para apostar no setor ...,Ex-sócios de peso da XP compraram uma fatia de...,2017-06-08,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...
88662,Posso doar as coisas dos outros?,A mãe de uma menina de sete anos contou que a ...,2015-09-05,colunas,quebracabeca,http://www1.folha.uol.com.br/colunas/quebracab...
12791,Comissão da UE propõe empréstimo de curto praz...,A Comissão Europeia está propondo conceder um ...,2015-07-15,mercado,NaN,http://www1.folha.uol.com.br/mercado/2015/07/1...
46378,Tegucigalpa,"- Uma água. R$2,50. Mais alguma coisa, senhor?...",2015-10-18,colunas,antonioprata,http://www1.folha.uol.com.br/colunas/antoniopr...
85649,Acidente com avião militar em base aérea na Es...,A queda de um avião militar F-16 da Grécia dei...,2015-01-26,mundo,NaN,http://www1.folha.uol.com.br/mundo/2015/01/158...


# Gerando Modelo

In [8]:
texto = "Rio de Janeiro é uma cidade maravilhosa"
doc = nlp(texto)

In [9]:
doc

Rio de Janeiro é uma cidade maravilhosa

In [10]:
type(doc[2])

spacy.tokens.token.Token

In [11]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino["title"])

In [12]:
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)

    if len(tokens_validos) > 2:
        return  " ".join(tokens_validos)

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)

In [13]:
texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [14]:
from time import time

t0 = time()
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                        batch_size = 1000,
                                                        n_process = -1)]

tf = time() - t0

print(tf/60)

3.28319814602534


In [15]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron e le pen a o turno frança revés siglas ...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço e alckmin anuncia que...
4,queda maio a atividade econômica sobe junho bc


In [16]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

In [17]:
w2v_modelo

In [18]:
print(len(titulos_tratados))
titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados))

90000
86113


In [19]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [20]:
import logging

logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

2022-03-07 22:38:31,840 : - collecting all words and their counts
2022-03-07 22:38:31,843 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-03-07 22:38:31,866 : - PROGRESS: at sentence #5000, processed 34716 words, keeping 10129 word types
2022-03-07 22:38:31,882 : - PROGRESS: at sentence #10000, processed 69298 words, keeping 14909 word types
2022-03-07 22:38:31,897 : - PROGRESS: at sentence #15000, processed 103841 words, keeping 18223 word types
2022-03-07 22:38:31,913 : - PROGRESS: at sentence #20000, processed 138620 words, keeping 20969 word types
2022-03-07 22:38:31,930 : - PROGRESS: at sentence #25000, processed 173257 words, keeping 23410 word types
2022-03-07 22:38:31,952 : - PROGRESS: at sentence #30000, processed 207976 words, keeping 25453 word types
2022-03-07 22:38:31,971 : - PROGRESS: at sentence #35000, processed 242567 words, keeping 27263 word types
2022-03-07 22:38:31,992 : - PROGRESS: at sentence #40000, processed 277254 words, keeping 2899

In [21]:
dir(w2v_modelo)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_input_data_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_job_params',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_minimize_model',
 '_raw_word_count',
 '_save_specials',
 '_set_train_params',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_update_job_params',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'accuracy',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'ca

In [22]:
w2v_modelo.corpus_count

86113

In [23]:
w2v_modelo.train(lista_lista_tokens, 
                 total_examples=w2v_modelo.corpus_count,
                 epochs = 30)

2022-03-07 22:38:35,518 : - training model with 3 workers on 13006 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2
2022-03-07 22:38:36,561 : - EPOCH 1 - PROGRESS: at 56.93% examples, 285033 words/s, in_qsize 5, out_qsize 0
2022-03-07 22:38:37,146 : - worker thread finished; awaiting finish of 2 more threads
2022-03-07 22:38:37,181 : - worker thread finished; awaiting finish of 1 more threads
2022-03-07 22:38:37,184 : - worker thread finished; awaiting finish of 0 more threads
2022-03-07 22:38:37,187 : - EPOCH - 1 : training on 597929 raw words (502902 effective words) took 1.6s, 308979 effective words/s
2022-03-07 22:38:38,254 : - EPOCH 2 - PROGRESS: at 61.95% examples, 297646 words/s, in_qsize 5, out_qsize 0
2022-03-07 22:38:38,749 : - worker thread finished; awaiting finish of 2 more threads
2022-03-07 22:38:38,770 : - worker thread finished; awaiting finish of 1 more threads
2022-03-07 22:38:38,778 : - worker thread finished; awaiting finish of 0 more t

(15086653, 17937870)

In [24]:
w2v_modelo.wv.most_similar("google")

2022-03-07 22:39:20,116 : - precomputing L2-norms of word weight vectors


[('apple', 0.5769424438476562),
 ('facebook', 0.551522433757782),
 ('uber', 0.4859882593154907),
 ('amazon', 0.48066601157188416),
 ('software', 0.4779198169708252),
 ('walmart', 0.4597754180431366),
 ('volkswagen', 0.45836687088012695),
 ('snapchat', 0.45630669593811035),
 ('yahoo', 0.45258331298828125),
 ('waze', 0.44571492075920105)]

In [25]:
w2v_modelo.wv.most_similar("microsoft")

[('amazon', 0.5997419953346252),
 ('unilever', 0.5724723935127258),
 ('tesla', 0.5651404857635498),
 ('sony', 0.5616632699966431),
 ('sky', 0.5525799989700317),
 ('canais', 0.5414109826087952),
 ('walmart', 0.5260538458824158),
 ('ford', 0.520609974861145),
 ('lego', 0.5149815678596497),
 ('braskem', 0.5143789649009705)]

In [26]:
w2v_modelo.wv.most_similar("barcelona")

[('psg', 0.6068384647369385),
 ('barça', 0.5947757959365845),
 ('bayern', 0.5686943531036377),
 ('chelsea', 0.567247748374939),
 ('juventus', 0.5637489557266235),
 ('united', 0.5602964162826538),
 ('figueirense', 0.5567421913146973),
 ('leicester', 0.5475366115570068),
 ('monaco', 0.5414623022079468),
 ('madrid', 0.539014458656311)]

In [27]:
w2v_modelo.wv.most_similar("messi")

[('suárez', 0.5877499580383301),
 ('neymar', 0.555770754814148),
 ('cristiano', 0.5375217795372009),
 ('tevez', 0.5229377746582031),
 ('ronaldo', 0.5074933767318726),
 ('cavani', 0.5020517110824585),
 ('barcelona', 0.49741482734680176),
 ('benzema', 0.4915468096733093),
 ('chuteiras', 0.48316478729248047),
 ('enrique', 0.483090877532959)]

In [28]:
w2v_modelo.wv.most_similar("gm")

[('chrysler', 0.6860479116439819),
 ('volks', 0.646731972694397),
 ('honda', 0.6461108922958374),
 ('toyota', 0.6358920931816101),
 ('volkswagen', 0.6198612451553345),
 ('embraer', 0.6157119274139404),
 ('renault', 0.6105237007141113),
 ('braskem', 0.6017100811004639),
 ('tesla', 0.587050199508667),
 ('sony', 0.5863809585571289)]

In [29]:
#Treinamento do modelo Skip-Gram
w2v_modelo_sg = Word2Vec(sg = 1,
                      window = 5,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)

w2v_modelo_sg.train(lista_lista_tokens, 
                 total_examples=w2v_modelo_sg.corpus_count,
                 epochs = 30)

2022-03-07 22:39:20,300 : - collecting all words and their counts
2022-03-07 22:39:20,305 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-03-07 22:39:20,327 : - PROGRESS: at sentence #5000, processed 34716 words, keeping 10129 word types
2022-03-07 22:39:20,349 : - PROGRESS: at sentence #10000, processed 69298 words, keeping 14909 word types
2022-03-07 22:39:20,372 : - PROGRESS: at sentence #15000, processed 103841 words, keeping 18223 word types
2022-03-07 22:39:20,394 : - PROGRESS: at sentence #20000, processed 138620 words, keeping 20969 word types
2022-03-07 22:39:20,411 : - PROGRESS: at sentence #25000, processed 173257 words, keeping 23410 word types
2022-03-07 22:39:20,428 : - PROGRESS: at sentence #30000, processed 207976 words, keeping 25453 word types
2022-03-07 22:39:20,445 : - PROGRESS: at sentence #35000, processed 242567 words, keeping 27263 word types
2022-03-07 22:39:20,463 : - PROGRESS: at sentence #40000, processed 277254 words, keeping 2899

(15087391, 17937870)

In [30]:
w2v_modelo_sg.wv.most_similar("google")

2022-03-07 22:41:11,749 : - precomputing L2-norms of word weight vectors


[('reguladores', 0.45178472995758057),
 ('apple', 0.4187764823436737),
 ('android', 0.3953913748264313),
 ('buffett', 0.3885148763656616),
 ('yahoo', 0.38758838176727295),
 ('anunciantes', 0.37553176283836365),
 ('verizon', 0.37003111839294434),
 ('automóveis', 0.3609682023525238),
 ('waze', 0.35871025919914246),
 ('intel', 0.356596440076828)]

In [31]:
w2v_modelo.wv.most_similar("google")

[('apple', 0.5769424438476562),
 ('facebook', 0.551522433757782),
 ('uber', 0.4859882593154907),
 ('amazon', 0.48066601157188416),
 ('software', 0.4779198169708252),
 ('walmart', 0.4597754180431366),
 ('volkswagen', 0.45836687088012695),
 ('snapchat', 0.45630669593811035),
 ('yahoo', 0.45258331298828125),
 ('waze', 0.44571492075920105)]

In [32]:
w2v_modelo_sg.wv.most_similar("gm")

[('metalúrgicos', 0.5842881202697754),
 ('motors', 0.5222570896148682),
 ('audi', 0.5042167901992798),
 ('cubatão', 0.4997138977050781),
 ('honda', 0.4948770999908447),
 ('bmw', 0.4737921357154846),
 ('mitsubishi', 0.4659489393234253),
 ('fiat', 0.4644043743610382),
 ('autoguiados', 0.46154266595840454),
 ('airbag', 0.45815417170524597)]

In [33]:
w2v_modelo.wv.most_similar("gm")

[('chrysler', 0.6860479116439819),
 ('volks', 0.646731972694397),
 ('honda', 0.6461108922958374),
 ('toyota', 0.6358920931816101),
 ('volkswagen', 0.6198612451553345),
 ('embraer', 0.6157119274139404),
 ('renault', 0.6105237007141113),
 ('braskem', 0.6017100811004639),
 ('tesla', 0.587050199508667),
 ('sony', 0.5863809585571289)]

In [34]:
w2v_modelo.wv.save_word2vec_format("modelo_cbow.txt", binary=False)
w2v_modelo_sg.wv.save_word2vec_format("modelo_skipgram.txt", binary=False)

2022-03-07 22:41:11,847 : - storing 13006x300 projection weights into modelo_cbow.txt
2022-03-07 22:41:14,613 : - storing 13006x300 projection weights into modelo_skipgram.txt


# Vetorizando treino e teste

In [35]:
import numpy as np
from gensim.models import KeyedVectors

In [36]:
w2v_modelo_cbow = KeyedVectors.load_word2vec_format("modelo_cbow.txt")
w2v_modelo_sg = KeyedVectors.load_word2vec_format("modelo_skipgram.txt")
artigo_treino = dados_treino
artigo_teste = dados_teste

2022-03-07 22:41:17,334 : - loading projection weights from modelo_cbow.txt
2022-03-07 22:41:19,595 : - loaded (13006, 300) matrix from modelo_cbow.txt
2022-03-07 22:41:19,596 : - loading projection weights from modelo_skipgram.txt
2022-03-07 22:41:21,928 : - loaded (13006, 300) matrix from modelo_skipgram.txt


In [37]:
nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])

def tokenizador(texto):
    
    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())

    
    return  tokens_validos

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
tokens = tokenizador(texto)
print(tokens)

['rio', 'janeiro', 'cidade', 'maravilhosa']


In [38]:
def combinacao_de_vetores_por_soma(palavras, modelo):

    vetor_resultante = np.zeros((1,300))

    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)

        except KeyError:
            pass
                

    return vetor_resultante

vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
print(vetor_texto.shape)
print(vetor_texto)

(1, 300)
[[-5.39423032e-02  2.05666050e-01  5.61055720e-01  7.94501007e-02
   7.88361818e-01 -1.98687884e+00  1.09880002e+00  8.35407436e-01
  -1.67026201e+00 -3.02382525e-01 -4.03450876e-01 -2.71810845e+00
  -9.40536499e-01  1.94282454e+00  6.00015432e-01  4.71153766e-01
   1.54639032e+00  1.08005215e-01 -3.11272196e-01 -2.15721226e+00
  -6.72993600e-01 -4.67150547e-01  2.10443613e+00 -2.01361774e+00
  -2.52699547e-01 -1.93549949e+00 -2.44662535e+00  1.20902634e+00
   4.26059009e-01 -2.28288782e+00 -1.55009404e-02  2.13869339e+00
  -9.68052596e-02  1.52843118e+00  7.68015465e-01  3.56920719e-01
   7.37850472e-01  1.54942555e+00  8.12472098e-01  9.01347056e-01
   1.56365925e+00 -5.51601946e-02  1.20413458e+00 -1.44400117e+00
   1.24045267e+00  1.55836654e+00  7.30244085e-01 -3.65597445e+00
   1.58766720e+00 -2.14718971e-01 -7.61160731e-01 -3.99800575e+00
  -6.13810629e-01 -1.15358061e+00 -8.61807615e-01 -1.85132340e+00
   2.21819620e+00 -9.86089051e-01 -6.85825005e-01 -5.46040151e-01
 

In [39]:
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

    return matriz

matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo_cbow)
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape) 

(90000, 300)
(20513, 300)


# Treino e integração classificador

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino, x_teste, y_teste):

    RL = LogisticRegression(max_iter = 800)
    RL.fit(x_treino, y_treino)
    categorias = RL.predict(x_teste)
    resultados = classification_report(y_teste, categorias)
    print(resultados)
    
    return RL

RL_cbow = classificador(w2v_modelo_cbow,
                        matriz_vetores_treino_cbow,
                        artigo_treino.category,
                        matriz_vetores_teste_cbow,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.63      0.80      0.70      1698
     esporte       0.92      0.86      0.89      4663
   ilustrada       0.13      0.86      0.23       131
     mercado       0.83      0.78      0.81      5867
       mundo       0.74      0.83      0.78      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [41]:
matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)

RL_sg = classificador(w2v_modelo_sg,
                        matriz_vetores_treino_sg,
                        artigo_treino.category,
                        matriz_vetores_teste_sg,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.93      0.87      0.90      4663
   ilustrada       0.14      0.85      0.24       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.75      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.69      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [42]:
import pickle

with open("rl_cbow.pkl", "wb") as f:
    pickle.dump(RL_cbow, f)

In [43]:
with open("rl_sg.pkl", "wb") as f:
    pickle.dump(RL_sg, f)